In [5]:
import sqlite3
import pandas as pd
from ipywidgets import Dropdown, Output, VBox, GridspecLayout
from IPython.display import display
from pathlib import Path
import ast
import sys

sys.path.insert(0, str(Path.cwd().parent))
from util.sql import refresh
from util.py3 import from_xyz_block, visualize_traj
from util.mpl import visualize_scan_energy

# Refresh database
refresh()

# Connect to database
db = Path.home() / "C5O-Kinetics/db/data.db"
conn = sqlite3.connect(db)


# Callback
def on_smiles_change(change):
    output[0, 0].clear_output()
    output[0, 1].clear_output()
    output[1, 0].clear_output()
    output[1, 1].clear_output()
    smiles_id = change["new"]

    # Query main data
    df_calculations = pd.read_sql_query(
        f"SELECT calc_id, method_id, scan_idx1, scan_idx2 FROM Calculations WHERE smiles_id = {smiles_id}",
        conn,
    )
    methods_opts = []
    for row in df_calculations.itertuples():
        df_methods = pd.read_sql_query(
            f"SELECT * FROM Methods WHERE method_id = {int(row.method_id)}",
            conn,
        )
        scan = (
            f"Scan {row.scan_idx1}:{row.scan_idx2}"
            if row.scan_idx1 != -1 and row.scan_idx2 != -1
            else "Original"
        )
        methods_opts.append(
            (
                f"{scan:<12} | {df_methods.iloc[0]['method']:<15}  | {df_methods.iloc[0]['functional']:<15} {df_methods.iloc[0]['basis']}",
                row.calc_id,
            )
        )
    calculations_dropdown.options = sorted(methods_opts)


def on_calculations_change(change):
    df_xyzs = pd.read_sql_query(
        f"SELECT xyz_text FROM xyz WHERE calc_id = {calculations_dropdown.value}",
        conn,
    )
    df_traj = pd.read_sql_query(
        f"SELECT traj_text FROM traj WHERE calc_id = {calculations_dropdown.value}",
        conn,
    )
    df_energy = pd.read_sql_query(
        f"SELECT * FROM energies WHERE calc_id = {calculations_dropdown.value}",
        conn,
    )
    output[0, 0].clear_output()
    with output[0, 0]:
        from_xyz_block(df_xyzs.iloc[0, 0])
    output[0, 1].clear_output()
    with output[0, 1]:
        visualize_traj(df_traj.iloc[0, 0])
    output[1, 1].clear_output()
    with output[1, 1]:
        if df_energy.iloc[0, 2]:
            arr = ast.literal_eval(df_energy.iloc[0, 2])
            steps, energies = arr
            visualize_scan_energy(steps=steps, energies=energies)
        else:
            display("Doesn't have energy")


# Options
df_smiles = pd.read_sql_query("SELECT smiles_id, smiles_text FROM SMILES", conn)
smiles_dict = dict(df_smiles.values)  # {smiles_id: smiles}
smiles_dropdown = Dropdown(
    options=[(row.smiles_text, row.smiles_id) for row in df_smiles.itertuples()],
    value=None,
    description="SMILES:",
)
smiles_dropdown.observe(on_smiles_change, names="value")
calculations_dropdown = Dropdown(options=[], description="Calculations:")
calculations_dropdown.observe(on_calculations_change, names="value")

# Outputs
output = GridspecLayout(2, 2)
output[0, 0] = Output()
output[0, 1] = Output()
output[1, 0] = Output()
output[1, 1] = Output()


# Display UI
display(VBox([smiles_dropdown, calculations_dropdown, output]))